In [ ]:
#importing neccessary libraries
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/processed_dataset (2).csv')

In [ ]:
df.head()

,Year of Project,Industry Sector,Application Type,Development Type,Development Platform,Language Type,Primary Programming Language,Count Approach,Functional Size,Relative Size,Summary Work Effort,Normalised PDR (ufp),Project Elapsed Time,Max Team Size,Development Methodologies,FP Standard,Package Customisation,Total Project Cost (USD)
0,2015,2,376,0,4,1,79,6,67.0,3,608.0,11.1,4.4,6.0,34,49,1,51311.800081
1,2016,2,142,0,4,1,79,6,51.0,3,288.0,6.2,1.0,6.0,34,49,1,32217.932055
2,1996,3,492,3,4,2,19,6,443.0,2,796.0,1.9,2.6,5.0,34,0,1,30488.742753
3,2002,22,25,0,2,1,41,6,76.0,3,1100.0,14.5,4.0,6.0,34,45,1,55728.768217
4,2004,2,219,0,4,1,79,6,3.0,8,28.0,9.3,6.0,6.0,34,30,1,4666.998461


In [ ]:
df.shape

(8473, 18)

In [ ]:
df.columns

Index(['Year of Project', 'Industry Sector', 'Application Type',
       'Development Type', 'Development Platform', 'Language Type',
       'Primary Programming Language', 'Count Approach', 'Functional Size',
       'Relative Size', 'Summary Work Effort', 'Normalised PDR (ufp)',
       'Project Elapsed Time', 'Max Team Size', 'Development Methodologies',
       'FP Standard', 'Package Customisation', 'Total Project Cost (USD)'],
      dtype='object')

In [ ]:
!pip install scikeras

  Using cached keras-3.4.1-py3-none-any.whl (1.1 MB)
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.4.1 which is incompatible.


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from keras.optimizers import RMSprop
from keras.layers import LeakyReLU, ELU, PReLU
from keras.layers import Dense, Dropout, Activation
from keras.models import Sequential
from keras.regularizers import l1, l2
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from scikeras.wrappers import KerasRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import pandas as pd

Feed-Forward Neural Network

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LeakyReLU
from scikeras.wrappers import KerasRegressor
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.regularizers import l1_l2
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Separate features and target variable
X = df.drop("Total Project Cost (USD)", axis=1)
y = df["Total Project Cost (USD)"]

# Normalize features
scaler_X = StandardScaler()
X = scaler_X.fit_transform(X)
X = pd.DataFrame(X)

X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.2, random_state=42)

# Define the function to create the Keras model
def create_model(learning_rate=0.001, neurons=64, dropout_rate=0.5, activation='leakyrelu', optimizer='adam', regularization=None):
    model = Sequential()
    if regularization == 'l1_l2':
        reg = l1_l2(l1=0.01, l2=0.01)
    else:
        reg = None

    model.add(Dense(neurons, input_shape=(X_train.shape[1],), kernel_regularizer=reg))
    if activation == 'leakyrelu':
        model.add(LeakyReLU(alpha=0.01))
    else:
        model.add(Activation(activation))
    model.add(Dropout(dropout_rate))

    model.add(Dense(neurons, kernel_regularizer=reg))
    if activation == 'leakyrelu':
        model.add(LeakyReLU(alpha=0.01))
    else:
        model.add(Activation(activation))
    model.add(Dropout(dropout_rate))

    model.add(Dense(neurons // 2, kernel_regularizer=reg))
    if activation == 'leakyrelu':
        model.add(LeakyReLU(alpha=0.01))
    else:
        model.add(Activation(activation))
    model.add(Dropout(dropout_rate))

    model.add(Dense(neurons // 2, kernel_regularizer=reg))
    if activation == 'leakyrelu':
        model.add(LeakyReLU(alpha=0.01))
    else:
        model.add(Activation(activation))
    model.add(Dense(1))

    if optimizer == 'rmsprop':
        opt = RMSprop(learning_rate=learning_rate)
    elif optimizer == 'adam':
        opt = Adam(learning_rate=learning_rate)

    model.compile(loss='mse', optimizer=opt, metrics=['mae'])
    return model

# Wrap the model using KerasRegressor
model = KerasRegressor(model=create_model, verbose=0)

# Define the hyperparameters to tune
param_dist = {
    'model__learning_rate': [0.0001, 0.001, 0.01, 0.1],
    'model__neurons': [32, 64, 128, 256],
    'model__dropout_rate': [0.2, 0.3, 0.5, 0.7],
    'model__activation': ['leakyrelu', 'relu', 'elu'],
    'model__optimizer': ['rmsprop', 'adam'],
    'model__regularization': [None, 'l1_l2'],
    'epochs': [50, 100, 200, 300],
    'batch_size': [16, 32, 64, 128]
}

# Use RandomizedSearchCV to find the best hyperparameters
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=20, cv=3, random_state=42, verbose=1, n_jobs=-1)
random_search_result = random_search.fit(X_train, y_train)

# Print the best hyperparameters
print(f"Best Hyperparameters: {random_search_result.best_params_}")

# Evaluate the best model on the test set
best_model = random_search_result.best_estimator_
y_pred = best_model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Calculate the average of the actual 'Total Project Cost (USD)'
average_actual_cost = np.mean(y_test)

# Calculate MAE and RMSE as percentages of the average actual cost
mae_percentage = (mae / average_actual_cost) * 100
rmse_percentage = (rmse / average_actual_cost) * 100

# Print results
print("MAE:", mae)
print("MAE as percentage of average actual cost:", mae_percentage)
print("Mean Squared Error:", mse)
print("RMSE:", rmse)
print("RMSE as percentage of average actual cost:", rmse_percentage)
print("R-squared:", r2)

Fitting 3 folds for each of 20 candidates, totalling 60 fits
Best Hyperparameters: {'model__regularization': None, 'model__optimizer': 'rmsprop', 'model__neurons': 256, 'model__learning_rate': 0.001, 'model__dropout_rate': 0.3, 'model__activation': 'leakyrelu', 'epochs': 100, 'batch_size': 16}
MAE: 37273.11280210122
MAE as percentage of average actual cost: 18.289836695150296
Mean Squared Error: 10852252254.077131
RMSE: 104174.14388454138
RMSE as percentage of average actual cost: 51.11802949277533
R-squared: 0.925278358690833


In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LeakyReLU
from scikeras.wrappers import KerasRegressor
from tensorflow.keras.optimizers import RMSprop
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV, KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# Load your dataset here (replace with your actual data loading code)
# For example:
# df = pd.read_csv('your_dataset.csv')

# Separate features and target variable
X = df.drop("Total Project Cost (USD)", axis=1)
y = df["Total Project Cost (USD)"]

# Normalize features
scaler_X = StandardScaler()
X = scaler_X.fit_transform(X)
X = pd.DataFrame(X)

# Split data into train and test sets
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.2, random_state=42)

# Define the function to create the Keras model
def create_model(learning_rate=0.001, neurons=256, dropout_rate=0.3): # no change here
    model = Sequential()
    model.add(Dense(neurons, input_shape=(X_train.shape[1],)))
    model.add(LeakyReLU(alpha=0.01))
    model.add(Dropout(dropout_rate))

    model.add(Dense(neurons))
    model.add(LeakyReLU(alpha=0.01))
    model.add(Dropout(dropout_rate))

    model.add(Dense(neurons // 2))
    model.add(LeakyReLU(alpha=0.01))
    model.add(Dropout(dropout_rate))

    model.add(Dense(neurons // 2))
    model.add(LeakyReLU(alpha=0.01))

    model.add(Dense(1))

    opt = RMSprop(learning_rate=learning_rate)
    model.compile(loss='mse', optimizer=opt, metrics=['mae'])
    return model

# Wrap the model using KerasRegressor, exposing 'neurons' as a tunable hyperparameter
model = KerasRegressor(model=create_model, learning_rate=0.001, neurons=256, dropout_rate=0.3, verbose=0)

# Define the hyperparameters to tune
param_dist = {
    'model__learning_rate': [0.0001, 0.001, 0.01],
    'model__neurons': [128, 256, 512], # Access the 'neurons' parameter through 'model__' prefix
    'model__dropout_rate': [0.2, 0.3, 0.5], # Access the 'dropout_rate' parameter through 'model__' prefix
    'epochs': [100],
    'batch_size': [16, 32, 64]
}

# Use KFold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Define callbacks for learning rate scheduling and early stopping
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


# Use RandomizedSearchCV with KFold cross-validation
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=10, cv=kf, scoring='neg_mean_absolute_error', verbose=1, random_state=42, n_jobs=-1)
random_search_result = random_search.fit(X_train, y_train, validation_data=(X_val, y_val), callbacks=[reduce_lr, early_stopping])

# Print the best hyperparameters
print(f"Best Hyperparameters: {random_search_result.best_params_}")

# Evaluate the best model on the test set
best_model = random_search_result.best_estimator_
y_pred = best_model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Printing Result
print("MAE:", mae)
print("Mean Squared Error:", mse)
print("RMSE:", rmse)
print("R-squared:", r2)


Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best Hyperparameters: {'model__neurons': 256, 'model__learning_rate': 0.001, 'model__dropout_rate': 0.5, 'epochs': 100, 'batch_size': 16}
MAE: 38690.73773656785
Mean Squared Error: 15051366702.83993
RMSE: 122684.0116023271
R-squared: 0.8963659526473119


In [ ]:
from keras.optimizers import RMSprop
from keras.layers import Dense, Dropout, Activation, LeakyReLU # Removed ThresholdedReLU import
from keras.models import Sequential
from keras.regularizers import l1, l2
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import pandas as pd

# Separate features and target variable
X = df.drop("Total Project Cost (USD)", axis=1)
y = df["Total Project Cost (USD)"]

scaler = StandardScaler()
X = scaler.fit_transform(X)
X = pd.DataFrame(X)

X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.2, random_state=42)

# Define the model
model1 = Sequential([
    Dense(128, input_shape=(X_train.shape[1],), activation=LeakyReLU(alpha=0.01)),
    Dropout(0.5),
    Dense(128, activation=LeakyReLU(alpha=0.01)),
    Dropout(0.5),
    Dense(64, activation=LeakyReLU(alpha=0.01)),
    Dropout(0.5),
    Dense(64, activation='relu'),  # Replace ThresholdedReLU with a standard ReLU activation
    Dense(1)
])

learning_rate = 0.001
optimizer = RMSprop(learning_rate)
model1.compile(loss='mse', optimizer=optimizer, metrics=['mae'])

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with early stopping for validation performance
model1.fit(X_train, y_train, epochs=500, validation_data=(X_val, y_val), verbose=0, callbacks=[early_stopping])

# Predict
y_pred1 = model1.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred1)
mse = mean_squared_error(y_test, y_pred1)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred1)

# Printing Result
print("MAE:", mae)
print("Mean Squared Error:", mse)
print("RMSE:", rmse)
print("R-squared:", r2)

53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
MAE: 49058.52355075877
Mean Squared Error: 19989859409.16255
RMSE: 141385.49928886822
R-squared: 0.862362662641669


In [ ]:
from keras.optimizers import RMSprop
from keras.layers import Dense, Dropout, Activation
from keras.models import Sequential
from keras.regularizers import l1, l2
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import ThresholdedReLU  # Assuming you have TensorFlow installed
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import pandas as pd

# Separate features and target variable
X = df.drop("Total Project Cost (USD)", axis=1)
y = df["Total Project Cost (USD)"]

# Normalize features
scaler_X = StandardScaler()
X = scaler_X.fit_transform(X)
X = pd.DataFrame(X)

X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.2, random_state=42)

# Define the model
model1 = Sequential([
    Dense(128, input_shape=(X_train.shape[1],), activation=LeakyReLU(alpha=0.01), kernel_regularizer=l2(0.01)),
    Dropout(0.5),
    Dense(128, activation=LeakyReLU(alpha=0.01), kernel_regularizer=l2(0.01)),
    Dropout(0.5),
    Dense(64, activation=LeakyReLU(alpha=0.01), kernel_regularizer=l2(0.01)),
    Dropout(0.5),
    Dense(64, activation=ThresholdedReLU(theta=1.0), kernel_regularizer=l2(0.01)),
    Dense(1)
])

learning_rate = 0.001
optimizer = RMSprop(learning_rate)
model1.compile(loss='mse', optimizer=optimizer, metrics=['mae'])

# Early stopping and learning rate reduction on plateau
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001)

# Train the model with early stopping and learning rate reduction on plateau
model1.fit(X_train, y_train, epochs=500, validation_data=(X_val, y_val), verbose=0, callbacks=[early_stopping, reduce_lr])

# Predict
y_pred1 = model1.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred1)
mse = mean_squared_error(y_test, y_pred1)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred1)

# Printing Result
print("MAE:", mae)
print("Mean Squared Error:", mse)
print("RMSE:", rmse)
print("R-squared:", r2)


53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
MAE: 50763.61174275873
Mean Squared Error: 20808879049.188763
RMSE: 144252.83029871117
R-squared: 0.8567234192538082


In [ ]:
from keras.optimizers import RMSprop, Adam
from keras.layers import Dense, Dropout, Activation
from keras.models import Sequential
from keras.regularizers import l1_l2
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import ThresholdedReLU  # Assuming you have TensorFlow installed
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import pandas as pd

# Separate features and target variable
X = df.drop("Total Project Cost (USD)", axis=1)
y = df["Total Project Cost (USD)"]

# Normalize features
scaler_X = StandardScaler()
X = scaler_X.fit_transform(X)
X = pd.DataFrame(X)

X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.2, random_state=42)

# Define the model
model1 = Sequential([
    Dense(128, input_shape=(X_train.shape[1],), activation=LeakyReLU(alpha=0.01), kernel_regularizer=l1_l2(l1=0.01, l2=0.01)),
    Dropout(0.5),
    Dense(128, activation=LeakyReLU(alpha=0.01), kernel_regularizer=l1_l2(l1=0.01, l2=0.01)),
    Dropout(0.5),
    Dense(64, activation=LeakyReLU(alpha=0.01), kernel_regularizer=l1_l2(l1=0.01, l2=0.01)),
    Dropout(0.5),
    Dense(64, activation=ThresholdedReLU(theta=1.0), kernel_regularizer=l1_l2(l1=0.01, l2=0.01)),
    Dense(1)
])

# Experiment with Adam optimizer
learning_rate = 0.001
optimizer = Adam(learning_rate)
model1.compile(loss='mse', optimizer=optimizer, metrics=['mae'])

# Early stopping and learning rate reduction on plateau
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001)

# Train the model with early stopping and learning rate reduction on plateau
model1.fit(X_train, y_train, epochs=500, validation_data=(X_val, y_val), verbose=0, callbacks=[early_stopping, reduce_lr])

# Predict
y_pred1 = model1.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred1)
mse = mean_squared_error(y_test, y_pred1)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred1)

# Printing Result
print("MAE:", mae)
print("Mean Squared Error:", mse)
print("RMSE:", rmse)
print("R-squared:", r2)

53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
MAE: 79743.67317726274
Mean Squared Error: 26194289744.135487
RMSE: 161846.5005619074
R-squared: 0.8196429389231765


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LeakyReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# Load your dataset here (replace with your actual data loading code)
# For example:
# df = pd.read_csv('your_dataset.csv')

# Separate features and target variable
X = df.drop("Total Project Cost (USD)", axis=1)
y = df["Total Project Cost (USD)"]

# Normalize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Define the model
model = Sequential([
    Dense(128, input_shape=(X_train.shape[1],)),
    LeakyReLU(alpha=0.01),
    Dropout(0.5),
    Dense(128),
    LeakyReLU(alpha=0.01),
    Dropout(0.5),
    Dense(64),
    LeakyReLU(alpha=0.01),
    Dropout(0.5),
    Dense(1)
])

# Compile the model with Adam optimizer and reduced learning rate
optimizer = Adam(learning_rate=0.001)
model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mae'])

# Define callbacks for learning rate scheduling and early stopping
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=16, validation_data=(X_val, y_val), callbacks=[reduce_lr, early_stopping])

# Predict on test set
y_pred = model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Printing Result
print("MAE:", mae)
print("Mean Squared Error:", mse)
print("RMSE:", rmse)
print("R-squared:", r2)


Epoch 1/100
339/339 ━━━━━━━━━━━━━━━━━━━━ 12s 20ms/step - loss: 5610297884672.0000 - mae: 279117.6875 - val_loss: 153197674496.0000 - val_mae: 170047.6719 - learning_rate: 0.0010
Epoch 2/100
339/339 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - loss: 1204625866752.0000 - mae: 161997.2656 - val_loss: 62796976128.0000 - val_mae: 87493.2656 - learning_rate: 0.0010
Epoch 3/100
339/339 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 4058653655040.0000 - mae: 143179.0312 - val_loss: 53351243776.0000 - val_mae: 85106.4609 - learning_rate: 0.0010
Epoch 4/100
339/339 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 856491753472.0000 - mae: 112218.8516 - val_loss: 48893833216.0000 - val_mae: 82245.5547 - learning_rate: 0.0010
Epoch 5/100
339/339 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1880391155712.0000 - mae: 114144.5000 - val_loss: 45729132544.0000 - val_mae: 79931.6484 - learning_rate: 0.0010
Epoch 6/100
339/339 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 320113442816.0000 - mae: 98598.5234 - val_loss: 43239391232.0000 

In [ ]:
from keras.optimizers import Adam
from keras.layers import Dense, Dropout, LeakyReLU, Activation # Import LeakyReLU
from keras.models import Sequential
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Separate features and target variable
X = df.drop("Total Project Cost (USD)", axis=1)
y = df["Total Project Cost (USD)"]

# Normalize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split data into train and test sets
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.2, random_state=42)

# Define the model
model = Sequential([
    Dense(256, input_shape=(X_train.shape[1],)),
    LeakyReLU(), # Use LeakyReLU layer instead of Activation
    Dropout(0.5),
    Dense(128),
    LeakyReLU(), # Use LeakyReLU layer instead of Activation
    Dropout(0.5),
    Dense(64),
    LeakyReLU(), # Use LeakyReLU layer instead of Activation
    Dropout(0.5),
    Dense(1)
])

# Compile the model with a lower learning rate
optimizer = Adam(learning_rate=0.001)
model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mae'])

# Train the model with early stopping for validation performance
model.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), verbose=0)

# Predict
y_pred = model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Printing Result
print("MAE:", mae)
print("Mean Squared Error:", mse)
print("RMSE:", rmse)
print("R-squared:", r2)


53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
MAE: 102346.46217496958
Mean Squared Error: 33341199565.939247
RMSE: 182595.72712946832
R-squared: 0.7704339065793926


In [ ]:
from keras.optimizers import Adam
from keras.layers import LeakyReLU, ELU, PReLU, ThresholdedReLU
from keras.layers import Dense, Dropout, Activation
from keras.models import Sequential
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.regularizers import l1, l2

# Separate features and target variable
X = df.drop("Total Project Cost (USD)", axis=1)
y = df["Total Project Cost (USD)"]

scaler = StandardScaler()
x = scaler.fit_transform(X)
X=pd.DataFrame(x)

X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.2, random_state=42)

# Define the model
model5 = Sequential([
    Dense(128, input_shape=(X_train.shape[1],), activation='relu', kernel_regularizer=l1(0.01)),  # L1 regularization
    Dropout(0.5),  # Dropout layer with 50% dropout rate
    BatchNormalization(),  # Batch normalization layer
    Dense(128, activation='LeakyReLU', kernel_regularizer=l2(0.01)),  # L2 regularization
    Dropout(0.5),  # Dropout layer with 50% dropout rate
    BatchNormalization(),  # Batch normalization layer
    Dense(64, activation='LeakyReLU', kernel_regularizer=l2(0.01)),  # L2 regularization
    Dropout(0.5),  # Dropout layer with 50% dropout rate
    BatchNormalization(),  # Batch normalization layer
    Dense(1, activation='ThresholdedReLU')
])

# Compile the model5 with a custom learning rate
optimizer = Adam(learning_rate=0.1)  # Experiment with different learning rates
model5.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mae'])
model5.summary()


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_25 (Dense)            (None, 128)               2304      
                                                                 
 dropout_10 (Dropout)        (None, 128)               0         
                                                                 
 batch_normalization (Batch  (None, 128)               512       
 Normalization)                                                  
                                                                 
 dense_26 (Dense)            (None, 128)               16512     
                                                                 
 dropout_11 (Dropout)        (None, 128)               0         
                                                                 
 batch_normalization_1 (Bat  (None, 128)               512       
 chNormalization)                                     

In [ ]:
model5.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val))

Epoch 1/100
170/170 [==============================] - 4s 9ms/step - loss: 2982637469696.0000 - mae: 223021.0156 - val_loss: 128560652288.0000 - val_mae: 167557.8438
Epoch 2/100
170/170 [==============================] - 1s 7ms/step - loss: 2957242007552.0000 - mae: 193844.7969 - val_loss: 81299013632.0000 - val_mae: 118851.1406
Epoch 3/100
170/170 [==============================] - 1s 7ms/step - loss: 2922173169664.0000 - mae: 160934.2031 - val_loss: 83207536640.0000 - val_mae: 145778.0938
Epoch 4/100
170/170 [==============================] - 1s 4ms/step - loss: 2894816346112.0000 - mae: 148768.1562 - val_loss: 40951029760.0000 - val_mae: 111275.8359
Epoch 5/100
170/170 [==============================] - 1s 4ms/step - loss: 2872010342400.0000 - mae: 145488.9531 - val_loss: 48211533824.0000 - val_mae: 82599.4844
Epoch 6/100
170/170 [==============================] - 1s 4ms/step - loss: 2868069793792.0000 - mae: 162092.5781 - val_loss: 33798014976.0000 - val_mae: 83685.6094
Epoch 7/100

In [ ]:
# Predict
y_pred5 = model5.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred5)
mse = mean_squared_error(y_test, y_pred5)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred5)

# Printing Result
print("MAE:", mae)
print("Mean Squared Error:", mse)
print("RMSE:", rmse)
print("R-squared:", r2)

53/53 [==============================] - 0s 2ms/step
MAE: 92710.16955768263
Mean Squared Error: 35767145529.36627
RMSE: 189122.0387193578
R-squared: 0.753730400259171


BPNN

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Separate features and target variable
X = df.drop("Total Project Cost (USD)", axis=1)
y = df["Total Project Cost (USD)"]

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Define the MLPRegressor
model = MLPRegressor(max_iter=1000, early_stopping=True, n_iter_no_change=50)

# Define the parameter grid
param_dist = {
    'hidden_layer_sizes': [(50,), (100,), (100, 50), (100, 100, 50)],
    'activation': ['relu', 'tanh'],
    'solver': ['adam', 'sgd'],
    'alpha': [0.0001, 0.001, 0.01],
    'learning_rate': ['constant', 'adaptive']
}

# Use RandomizedSearchCV for hyperparameter tuning
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=50, cv=3, scoring='neg_mean_absolute_error', verbose=2, random_state=42, n_jobs=-1)
random_search_result = random_search.fit(X_train, y_train)

# Print the best hyperparameters
print(f"Best Hyperparameters: {random_search_result.best_params_}")

# Evaluate the best model on the test set
best_model = random_search_result.best_estimator_
y_pred = best_model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Printing Result
print("MAE:", mae)
print("Mean Squared Error:", mse)
print("RMSE:", rmse)
print("R-squared:", r2)

Fitting 3 folds for each of 50 candidates, totalling 150 fits
Best Hyperparameters: {'solver': 'adam', 'learning_rate': 'constant', 'hidden_layer_sizes': (100, 100, 50), 'alpha': 0.01, 'activation': 'relu'}
MAE: 83038.192489451
Mean Squared Error: 28841263393.005005
RMSE: 169827.15740718562
R-squared: 0.8014175778723092


Radial Basis Neural Networks (RBFNN)

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Separate features and target variable
X = df.drop("Total Project Cost (USD)", axis=1)
y = df["Total Project Cost (USD)"]

scaler = StandardScaler()
x = scaler.fit_transform(X)
X=pd.DataFrame(x)

# Assuming X and y are your datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a pipeline with a scaler and MLPRegressor with RBF activation
model = make_pipeline(StandardScaler(), MLPRegressor(hidden_layer_sizes=(10,), activation='relu', solver='adam', max_iter=1000))

# Train the model
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Printing Result
print("MAE:", mae)
print("Mean Squared Error:", mse)
print("RMSE:", rmse)
print("R-squared:", r2)

MAE: 180186.20895521005
Mean Squared Error: 157829887479.49667
RMSE: 397278.0984140665
R-squared: -0.08671527015771896


In [ ]:
import numpy as np
import pandas as pd
from sklearn.kernel_approximation import RBFSampler
from sklearn.linear_model import Ridge
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import uniform

# Separate features and target variable
X = df.drop("Total Project Cost (USD)", axis=1)
y = df["Total Project Cost (USD)"]

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Define the RBFSampler and linear model pipeline
model = make_pipeline(RBFSampler(), Ridge())

# Define the parameter grid
param_dist = {
    'rbfsampler__gamma': uniform(0.1, 1),
    'ridge__alpha': uniform(0.1, 10)
}

# Use RandomizedSearchCV for hyperparameter tuning
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=50, cv=3, scoring='neg_mean_absolute_error', verbose=2, random_state=42, n_jobs=-1)
random_search_result = random_search.fit(X_train, y_train)

# Print the best hyperparameters
print(f"Best Hyperparameters: {random_search_result.best_params_}")

# Evaluate the best model on the test set
best_model = random_search_result.best_estimator_
y_pred = best_model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Printing Result
print("MAE:", mae)
print("Mean Squared Error:", mse)
print("RMSE:", rmse)
print("R-squared:", r2)


Fitting 3 folds for each of 50 candidates, totalling 150 fits
Best Hyperparameters: {'rbfsampler__gamma': 0.1055221171236024, 'ridge__alpha': 8.25461428454834}
MAE: 195414.9110495229
Mean Squared Error: 125397942299.48141
RMSE: 354115.71879751596
R-squared: 0.13659028135019258


Multilayer Perceptron Neural Network (MLPNN)

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Separate features and target variable
X = df.drop("Total Project Cost (USD)", axis=1)
y = df["Total Project Cost (USD)"]

scaler = StandardScaler()
x = scaler.fit_transform(X)
X=pd.DataFrame(x)

# Assuming X and y are your datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define and train the MLPNN model
mlp = MLPRegressor(hidden_layer_sizes=(100, 50), activation='relu', solver='adam', random_state=42)
mlp.fit(X_train, y_train)

# Make predictions
y_pred = mlp.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Printing Result
print("MAE:", mae)
print("Mean Squared Error:", mse)
print("RMSE:", rmse)
print("R-squared:", r2)


MAE: 89367.97210969104
Mean Squared Error: 34266223944.533813
RMSE: 185111.38253639027
R-squared: 0.7640647826223255


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Separate features and target variable
X = df.drop("Total Project Cost (USD)", axis=1)
y = df["Total Project Cost (USD)"]

scaler = StandardScaler()
x = scaler.fit_transform(X)
X=pd.DataFrame(x)

# Assuming X and y are your datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the MLP model
model = Sequential([
    Dense(128, activation='relu', input_dim=X_train.shape[1]),
    Dense(64, activation='relu'),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

y_pred = model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Printing Result
print("MAE:", mae)
print("Mean Squared Error:", mse)
print("RMSE:", rmse)
print("R-squared:", r2)

Epoch 1/100
170/170 [==============================] - 2s 5ms/step - loss: 861537107968.0000 - val_loss: 8672604848128.0000
Epoch 2/100
170/170 [==============================] - 1s 5ms/step - loss: 860866871296.0000 - val_loss: 8670846910464.0000
Epoch 3/100
170/170 [==============================] - 1s 4ms/step - loss: 858241302528.0000 - val_loss: 8666052820992.0000
Epoch 4/100
170/170 [==============================] - 1s 5ms/step - loss: 852473151488.0000 - val_loss: 8656803332096.0000
Epoch 5/100
170/170 [==============================] - 1s 5ms/step - loss: 842891001856.0000 - val_loss: 8643092676608.0000
Epoch 6/100
170/170 [==============================] - 1s 5ms/step - loss: 829835313152.0000 - val_loss: 8624940253184.0000
Epoch 7/100
170/170 [==============================] - 1s 5ms/step - loss: 814522105856.0000 - val_loss: 8604663939072.0000
Epoch 8/100
170/170 [==============================] - 1s 4ms/step - loss: 798279335936.0000 - val_loss: 8584148025344.0000
Epoch 9/

Convolutional Neural Networks (CNN)

In [ ]:
from tensorflow.keras.layers import Conv1D, Flatten, Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Separate features and target variable
X = df.drop("Total Project Cost (USD)", axis=1)
y = df["Total Project Cost (USD)"]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Assuming X and y are your datasets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Example with 1D Convolutional layers for sequence data
model = Sequential([
    Conv1D(32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
    Conv1D(16, kernel_size=3, activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')

# Note: Ensure X_train and X_test are reshaped for 1D convolutions
X_train = X_train.reshape(-1, X_train.shape[1], 1)
X_test = X_test.reshape(-1, X_test.shape[1], 1)

model.fit(X_train, y_train, epochs=50, batch_size=32)

y_pred = model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Printing Result
print("MAE:", mae)
print("Mean Squared Error:", mse)
print("RMSE:", rmse)
print("R-squared:", r2)

Epoch 1/50
212/212 [==============================] - 2s 3ms/step - loss: 2422535618560.0000
Epoch 2/50
212/212 [==============================] - 1s 3ms/step - loss: 2398682873856.0000
Epoch 3/50
212/212 [==============================] - 1s 4ms/step - loss: 2353439703040.0000
Epoch 4/50
212/212 [==============================] - 1s 4ms/step - loss: 2339388522496.0000
Epoch 5/50
212/212 [==============================] - 1s 5ms/step - loss: 2335675514880.0000
Epoch 6/50
212/212 [==============================] - 1s 5ms/step - loss: 2333160243200.0000
Epoch 7/50
212/212 [==============================] - 1s 4ms/step - loss: 2330928349184.0000
Epoch 8/50
212/212 [==============================] - 1s 3ms/step - loss: 2329098321920.0000
Epoch 9/50
212/212 [==============================] - 1s 2ms/step - loss: 2327164747776.0000
Epoch 10/50
212/212 [==============================] - 1s 3ms/step - loss: 2325531066368.0000
Epoch 11/50
212/212 [==============================] - 1s 3ms/step - 

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Separate features and target variable
X = df.drop("Total Project Cost (USD)", axis=1)
y = df["Total Project Cost (USD)"]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Assuming X and y are your datasets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Reshape data for Conv1D
X_train = X_train.reshape(-1, X_train.shape[1], 1)
X_test = X_test.reshape(-1, X_test.shape[1], 1)

# Model with additional layers and dropout
model = Sequential([
    Conv1D(64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
    BatchNormalization(),
    Dropout(0.2),
    Conv1D(32, kernel_size=3, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(1)
])

# Compile model with Adam optimizer and learning rate schedule
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mse')

# Early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with early stopping
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

# Predict and evaluate the model
y_pred = model.predict(X_test).flatten()

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Print results
print("MAE:", mae)
print("Mean Squared Error:", mse)
print("RMSE:", rmse)
print("R-squared:", r2)


Epoch 1/100
170/170 ━━━━━━━━━━━━━━━━━━━━ 16s 39ms/step - loss: 646933315584.0000 - val_loss: 8672623722496.0000
Epoch 2/100
170/170 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1155562340352.0000 - val_loss: 8671101714432.0000
Epoch 3/100
170/170 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 359062274048.0000 - val_loss: 8663127293952.0000
Epoch 4/100
170/170 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 547647913984.0000 - val_loss: 8641339457536.0000
Epoch 5/100
170/170 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 545814446080.0000 - val_loss: 8618458480640.0000
Epoch 6/100
170/170 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 216448761856.0000 - val_loss: 8581448990720.0000
Epoch 7/100
170/170 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 846315847680.0000 - val_loss: 8561966448640.0000
Epoch 8/100
170/170 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 560371204096.0000 - val_loss: 8539006304256.0000
Epoch 9/100
170/170 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 431734259712.0000 - val_loss: 8523519361024.0000
Epoch 1

Back-propagation Neural Network (BPNN)

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from sklearn.metrics import mean_squared_error


# Separate features and target variable
X = df.drop("Total Project Cost (USD)", axis=1)
y = df["Total Project Cost (USD)"]

scaler = StandardScaler()
x = scaler.fit_transform(X)
X=pd.DataFrame(x)

# Assuming X and y are your datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the BPNN model
model = Sequential()
model.add(Dense(100, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(1))

# Compile the model
model.compile(loss='mean_squared_error', optimizer=Adam())

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=1)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Printing Result
print("MAE:", mae)
print("Mean Squared Error:", mse)
print("RMSE:", rmse)
print("R-squared:", r2)

Epoch 1/100
212/212 [==============================] - 2s 3ms/step - loss: 2424234311680.0000
Epoch 2/100
212/212 [==============================] - 1s 4ms/step - loss: 2423390994432.0000
Epoch 3/100
212/212 [==============================] - 1s 3ms/step - loss: 2420536246272.0000
Epoch 4/100
212/212 [==============================] - 1s 3ms/step - loss: 2415031222272.0000
Epoch 5/100
212/212 [==============================] - 0s 2ms/step - loss: 2406655983616.0000
Epoch 6/100
212/212 [==============================] - 0s 2ms/step - loss: 2395452211200.0000
Epoch 7/100
212/212 [==============================] - 0s 2ms/step - loss: 2382117994496.0000
Epoch 8/100
212/212 [==============================] - 0s 2ms/step - loss: 2367070142464.0000
Epoch 9/100
212/212 [==============================] - 0s 2ms/step - loss: 2352255860736.0000
Epoch 10/100
212/212 [==============================] - 0s 2ms/step - loss: 2338932654080.0000
Epoch 11/100
212/212 [==============================] - 0s 

Functional Link Artificial Neural Network (FLANN)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import mean_squared_error

# Define function to create augmented features
def create_augmented_features(X):
    # Assuming X is a NumPy array
    X_poly = np.power(X, 2)  # Add squared terms
    X_sin = np.sin(X)        # Add sine transformed terms
    return np.concatenate([X, X_poly, X_sin], axis=1)

# Separate features and target variable
X = df.drop("Total Project Cost (USD)", axis=1)
y = df["Total Project Cost (USD)"]

# Augment features
X_augmented = create_augmented_features(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_augmented, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create the model
model = Sequential()
model.add(Dense(20, input_dim=X_train_scaled.shape[1], activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train_scaled, y_train, epochs=50, batch_size=10)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Printing Result
print("MAE:", mae)
print("Mean Squared Error:", mse)
print("RMSE:", rmse)
print("R-squared:", r2)

Epoch 1/50
678/678 [==============================] - 2s 2ms/step - loss: 2424237981696.0000
Epoch 2/50
678/678 [==============================] - 1s 2ms/step - loss: 2423601233920.0000
Epoch 3/50
678/678 [==============================] - 1s 2ms/step - loss: 2421616803840.0000
Epoch 4/50
678/678 [==============================] - 1s 2ms/step - loss: 2418032771072.0000
Epoch 5/50
678/678 [==============================] - 1s 2ms/step - loss: 2412843106304.0000
Epoch 6/50
678/678 [==============================] - 1s 2ms/step - loss: 2405907038208.0000
Epoch 7/50
678/678 [==============================] - 2s 3ms/step - loss: 2397908762624.0000
Epoch 8/50
678/678 [==============================] - 2s 3ms/step - loss: 2389095481344.0000
Epoch 9/50
678/678 [==============================] - 2s 2ms/step - loss: 2379040948224.0000
Epoch 10/50
678/678 [==============================] - 1s 2ms/step - loss: 2368357531648.0000
Epoch 11/50
678/678 [==============================] - 1s 2ms/step - 

Wavelet Neural Network (WNN)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.utils import get_custom_objects
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Define the custom Morlet wavelet function
def morlet_wavelet(x):
    return tf.multiply(tf.cos(1.75 * x), tf.exp(-0.5 * tf.square(x)))

# Register the custom activation function with Keras
get_custom_objects().update({'morlet_wavelet': morlet_wavelet})

# Define the wavelet neural network model
def build_wavelet_model(input_shape, neurons=64, dropout_rate=0.2, learning_rate=0.001):
    inputs = Input(shape=(input_shape,))
    x = Dense(neurons, activation=morlet_wavelet)(inputs)
    x = Dropout(dropout_rate)(x)
    x = Dense(neurons, activation=morlet_wavelet)(x)
    x = Dropout(dropout_rate)(x)
    outputs = Dense(1)(x)
    model = Model(inputs, outputs)
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    return model

# Separate features and target variable
X = df.drop("Total Project Cost (USD)", axis=1)
y = df["Total Project Cost (USD)"]
# Convert Series to numpy array if necessary
if isinstance(y, pd.Series):
    y = y.to_numpy()

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale inputs
scaler_x = StandardScaler()
X_train_scaled = scaler_x.fit_transform(X_train)
X_test_scaled = scaler_x.transform(X_test)

# If y needs scaling:
scaler_y = StandardScaler()
y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1))
y_test_scaled = scaler_y.transform(y_test.reshape(-1, 1))

# Build and train the model
model = build_wavelet_model(input_shape=X_train_scaled.shape[1], neurons=128, dropout_rate=0.3, learning_rate=0.001)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model.fit(X_train_scaled, y_train_scaled, epochs=100, batch_size=32, validation_split=0.2, callbacks=[early_stopping], verbose=1)

# Make predictions
y_pred_scaled = model.predict(X_test_scaled)

# Inverse transform the scaled predictions
y_pred = scaler_y.inverse_transform(y_pred_scaled)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Printing Result
print("MAE:", mae)
print("Mean Squared Error:", mse)
print("RMSE:", rmse)
print("R-squared:", r2)

Epoch 1/100
170/170 ━━━━━━━━━━━━━━━━━━━━ 11s 22ms/step - loss: 0.9294 - mae: 0.5500 - val_loss: 3.6386 - val_mae: 0.2358
Epoch 2/100
170/170 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3389 - mae: 0.2584 - val_loss: 3.6095 - val_mae: 0.1613
Epoch 3/100
170/170 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3185 - mae: 0.1776 - val_loss: 3.6025 - val_mae: 0.1458
Epoch 4/100
170/170 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4007 - mae: 0.1456 - val_loss: 3.6060 - val_mae: 0.1618
Epoch 5/100
170/170 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0938 - mae: 0.1134 - val_loss: 3.5976 - val_mae: 0.1459
Epoch 6/100
170/170 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0868 - mae: 0.0999 - val_loss: 3.5952 - val_mae: 0.1542
Epoch 7/100
170/170 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0763 - mae: 0.0952 - val_loss: 3.5954 - val_mae: 0.1357
Epoch 8/100
170/170 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0766 - mae: 0.0909 - val_loss: 3.6001 - val_mae: 0.1410
Epoch 9/100
170/170 ━━━━━━━━━━━━━━━━━━━━ 1s 3m

In [ ]:
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Define the wavelet activation function
def morlet_wavelet(x):
    return tf.multiply(tf.cos(1.75 * x), tf.exp(-0.5 * tf.square(x)))

# Register the custom activation with Keras
from tensorflow.keras.utils import get_custom_objects
get_custom_objects().update({'morlet_wavelet': morlet_wavelet})

def build_model(neurons=64, dropout_rate=0.2, learning_rate=0.001):
    inputs = Input(shape=(X_train_scaled.shape[1],))
    x = Dense(neurons, activation=morlet_wavelet)(inputs)
    x = Dropout(dropout_rate)(x)
    x = Dense(neurons, activation=morlet_wavelet)(x)
    x = Dropout(dropout_rate)(x)
    outputs = Dense(1)(x)
    model = Model(inputs, outputs)
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mse')
    return model

# Create KerasRegressor
model = KerasRegressor(
    model=build_model,
    verbose=0,
    neurons=64,
    dropout_rate=0.2,
    learning_rate=0.001,
    epochs=100,
    batch_size=32
)

# Define the parameter grid
param_distribs = {
    'model__neurons': [64, 128, 256],
    'model__dropout_rate': [0.2, 0.3, 0.5],
    'model__learning_rate': [0.001, 0.01, 0.1],
    'batch_size': [16, 32, 64],
    'epochs': [100, 200, 300]
}

# RandomizedSearchCV
rnd_search = RandomizedSearchCV(estimator=model, param_distributions=param_distribs, n_iter=20, cv=3, scoring='neg_mean_squared_error', verbose=2)
rnd_search.fit(X_train_scaled, y_train_scaled)

# Best parameters
print("Best Hyperparameters:", rnd_search.best_params_)

# Train the best model with early stopping
best_model = build_model(
    neurons=rnd_search.best_params_['model__neurons'],
    dropout_rate=rnd_search.best_params_['model__dropout_rate'],
    learning_rate=rnd_search.best_params_['model__learning_rate']
)

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
best_model.fit(
    X_train_scaled, y_train_scaled,
    epochs=rnd_search.best_params_['epochs'],
    batch_size=rnd_search.best_params_['batch_size'],
    validation_split=0.2,
    callbacks=[early_stopping],
    verbose=1
)

# Predict on the test set
y_pred = best_model.predict(X_test_scaled)

# Inverse transform predictions and actual values if they were scaled
y_pred = scaler_y.inverse_transform(y_pred)
y_test = scaler_y.inverse_transform(y_test_scaled)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Print results
print("MAE:", mae)
print("Mean Squared Error:", mse)
print("RMSE:", rmse)
print("R-squared:", r2)


Fitting 3 folds for each of 20 candidates, totalling 60 fits
[CV] END batch_size=32, epochs=200, model__dropout_rate=0.5, model__learning_rate=0.001, model__neurons=64; total time= 1.1min
[CV] END batch_size=32, epochs=200, model__dropout_rate=0.5, model__learning_rate=0.001, model__neurons=64; total time= 1.1min
[CV] END batch_size=32, epochs=200, model__dropout_rate=0.5, model__learning_rate=0.001, model__neurons=64; total time= 1.1min
[CV] END batch_size=16, epochs=100, model__dropout_rate=0.5, model__learning_rate=0.1, model__neurons=256; total time= 1.0min
[CV] END batch_size=16, epochs=100, model__dropout_rate=0.5, model__learning_rate=0.1, model__neurons=256; total time= 1.1min
[CV] END batch_size=16, epochs=100, model__dropout_rate=0.5, model__learning_rate=0.1, model__neurons=256; total time= 1.1min
[CV] END batch_size=64, epochs=100, model__dropout_rate=0.5, model__learning_rate=0.01, model__neurons=256; total time=  25.8s
[CV] END batch_size=64, epochs=100, model__dropout_ra

Generalized Regression Neural Networks (GRNN)

In [ ]:
!pip install pyGRNN

In [ ]:
import numpy as np
from pyGRNN import GRNN

# Separate features and target variable
X = df.drop("Total Project Cost (USD)", axis=1)
y = df["Total Project Cost (USD)"]

scaler = StandardScaler()
x = scaler.fit_transform(X)
X=pd.DataFrame(x)

# Assuming X and y are your datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train GRNN model
grnn = GRNN()
grnn.fit(X_train, y_train)

# Make predictions
y_pred = grnn.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Printing Result
print("MAE:", mae)
print("Mean Squared Error:", mse)
print("RMSE:", rmse)
print("R-squared:", r2)

Executing warm start...
Warm start concluded. The optimum isotropic sigma is [1.32948663]
Gradient search concluded. The optimum sigma is [3.63538044 1.80309565 4.67103031 1.65347581 6.27462607 5.68535867
 4.37694962 0.05898241 8.49173485 0.19032693 0.21216142 2.59337011
 4.16006863 1.66269481 0.09752468 1.39792372 1.329     ]
MAE: 51216.34727360432
Mean Squared Error: 23242833837.446236
RMSE: 152456.00623604908
R-squared: 0.8399647693078879


OLSRNN (Output Layer Self-Recurrent Neural Network)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import GlorotUniform

# Separate features and target variable
X = df.drop("Total Project Cost (USD)", axis=1)
y = df["Total Project Cost (USD)"]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize data
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Use a smaller learning rate
optimizer = Adam(learning_rate=0.0000001)

# Use a better initializer
initializer = GlorotUniform()

model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1] + 1,), kernel_initializer=initializer),
    keras.layers.Dense(64, activation='relu', kernel_initializer=initializer),
    keras.layers.Dense(1, kernel_initializer=initializer)
])

model.compile(optimizer=optimizer, loss='mean_squared_error')

# Define the loss function
loss_fn = tf.keras.losses.MeanSquaredError()

# Training parameters
epochs = 100
batch_size = 4

# Train the model with custom loop
for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))
    # Initialize feedback for each epoch
    output_feedback = np.zeros((batch_size, 1))  # Make sure feedback is correctly sized

    for step in range(len(X_train_scaled) // batch_size):
        # Select batch
        batch_X = X_train_scaled[step * batch_size:(step + 1) * batch_size]
        batch_y = y_train[step * batch_size:(step + 1) * batch_size]

        # Handle last batch which might be smaller than the batch_size
        actual_batch_size = batch_X.shape[0]
        if actual_batch_size < batch_size:
            output_feedback = np.zeros((actual_batch_size, 1))

        # Concatenate the output feedback
        batch_X_with_feedback = np.hstack([batch_X, output_feedback])

        with tf.GradientTape() as tape:
            # Run the model
            logits = model(batch_X_with_feedback, training=True)

            # Save output for feedback
            output_feedback = logits.numpy()

            # Compute the loss
            loss_value = loss_fn(batch_y, logits)

        # Update weights
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        print("Training loss (for one batch) at step %d: %.4f" % (step, float(loss_value)))

# Evaluate the model
X_test_with_feedback = np.hstack([X_test_scaled, np.zeros((len(X_test_scaled), 1))])  # No feedback when testing
predictions = model.predict(X_test_with_feedback)

# Evaluate the model
mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, predictions)

# Printing Result
print("MAE:", mae)
print("Mean Squared Error:", mse)
print("RMSE:", rmse)
print("R-squared:", r2)

Streaming output truncated to the last 5000 lines.
Training loss (for one batch) at step 666: 620142133248.0000
Training loss (for one batch) at step 667: 35356934144.0000
Training loss (for one batch) at step 668: 27899721728.0000
Training loss (for one batch) at step 669: 20161576960.0000
Training loss (for one batch) at step 670: 1441821440.0000
Training loss (for one batch) at step 671: 26617556992.0000
Training loss (for one batch) at step 672: 164070948864.0000
Training loss (for one batch) at step 673: 4900618240.0000
Training loss (for one batch) at step 674: 10829395968.0000
Training loss (for one batch) at step 675: 54399176704.0000
Training loss (for one batch) at step 676: 70174892032.0000
Training loss (for one batch) at step 677: 10124705792.0000
Training loss (for one batch) at step 678: 26612633600.0000
Training loss (for one batch) at step 679: 14348090368.0000
Training loss (for one batch) at step 680: 2751237120.0000
Training loss (for one batch) at step 681: 4914039

Spiking Neural Network (SNN)

In [ ]:
!pip install norse

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 6.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Us

In [ ]:
import torch
from torch import nn
from norse.torch import LIFCell, LIFState  # Ensure correct imports for LIFCell and LIFState
from torch.optim import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Separate features and target variable
X = df.drop("Total Project Cost (USD)", axis=1)
y = df["Total Project Cost (USD)"]

# Convert Series to numpy array if necessary
if isinstance(y, pd.Series):
    y = y.to_numpy()

# Splitting dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardizing the features
scaler = StandardScaler()
X_train_scaled = torch.tensor(scaler.fit_transform(X_train), dtype=torch.float32)
X_test_scaled = torch.tensor(scaler.transform(X_test), dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)  # Make y_train a column vector
y_test = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)    # Make y_test a column vector

# Define the SNN model
class SNNRegressor(nn.Module):
    def __init__(self, input_size):
        super(SNNRegressor, self).__init__()
        self.lif1 = LIFCell()
        self.fc = nn.Linear(input_size, 1)  # Output layer

    def forward(self, x):
        batch_size, seq_len, _ = x.size()
        s = torch.zeros(batch_size, 1)
        v = torch.zeros(batch_size, 1)
        z = torch.zeros(batch_size, 1)  # Initialize adaptation variable `z` to zero
        state = LIFState(v=v, i=s, z=z)  # Add the missing `z` parameter

        for t in range(seq_len):
            out, state = self.lif1(x[:, t], state)

        out = self.fc(state.v)
        return out

# Adjust the input shape to match LIFCell
X_train_scaled = X_train_scaled.unsqueeze(1)  # Add a dimension for sequence length
X_test_scaled = X_test_scaled.unsqueeze(1)    # Add a dimension for sequence length

# Training the SNN
input_size = X_train_scaled.shape[2]
model = SNNRegressor(input_size=input_size)
criterion = nn.MSELoss()
optimizer = Adam(model.parameters(), lr=0.02)

# Train the model
for epoch in range(100):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_scaled)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch}, Loss {loss.item()}")

# Evaluation
model.eval()
with torch.no_grad():
    predictions = model(X_test_scaled)

    # Convert tensors to NumPy arrays for compatibility with scikit-learn
    y_test_np = y_test.numpy()
    predictions_np = predictions.numpy()

    # Evaluate the model
    mae = mean_absolute_error(y_test_np, predictions_np)
    mse = mean_squared_error(y_test_np, predictions_np)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test_np, predictions_np)

    # Printing Result
    print("MAE:", mae)
    print("Mean Squared Error:", mse)
    print("RMSE:", rmse)
    print("R-squared:", r2)

Epoch 0, Loss 2424271798272.0
Epoch 1, Loss 2424271798272.0
Epoch 2, Loss 2424271798272.0
Epoch 3, Loss 2424271798272.0
Epoch 4, Loss 2424271798272.0
Epoch 5, Loss 2424271798272.0
Epoch 6, Loss 2424271798272.0
Epoch 7, Loss 2424271798272.0
Epoch 8, Loss 2424271536128.0
Epoch 9, Loss 2424271536128.0
Epoch 10, Loss 2424271536128.0
Epoch 11, Loss 2424271536128.0
Epoch 12, Loss 2424271536128.0
Epoch 13, Loss 2424271536128.0
Epoch 14, Loss 2424271536128.0
Epoch 15, Loss 2424271536128.0
Epoch 16, Loss 2424271536128.0
Epoch 17, Loss 2424271536128.0
Epoch 18, Loss 2424271536128.0
Epoch 19, Loss 2424271536128.0
Epoch 20, Loss 2424271536128.0
Epoch 21, Loss 2424271536128.0
Epoch 22, Loss 2424271536128.0
Epoch 23, Loss 2424271536128.0
Epoch 24, Loss 2424271536128.0
Epoch 25, Loss 2424271536128.0
Epoch 26, Loss 2424271536128.0
Epoch 27, Loss 2424271536128.0
Epoch 28, Loss 2424271536128.0
Epoch 29, Loss 2424271536128.0
Epoch 30, Loss 2424271536128.0
Epoch 31, Loss 2424271536128.0
Epoch 32, Loss 242

In [ ]:
import torch
from torch import nn
from norse.torch import LIFCell, LIFState  # Ensure correct imports for LIFCell and LIFState
from torch.optim import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Load your dataset `df`
# Assuming df is your pandas DataFrame with appropriate columns

# Separate features and target variable
X = df.drop("Total Project Cost (USD)", axis=1)
y = df["Total Project Cost (USD)"]

# Convert Series to numpy array if necessary
if isinstance(y, pd.Series):
    y = y.to_numpy()

# Splitting dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardizing the features
scaler = StandardScaler()
X_train_scaled = torch.tensor(scaler.fit_transform(X_train), dtype=torch.float32)
X_test_scaled = torch.tensor(scaler.transform(X_test), dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)  # Make y_train a column vector
y_test = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)    # Make y_test a column vector

# Define the SNN model with enhanced architecture
class EnhancedSNNRegressor(nn.Module):
    def __init__(self, input_size):
        super(EnhancedSNNRegressor, self).__init__()
        self.lif1 = LIFCell()
        self.lif2 = LIFCell()
        self.fc1 = nn.Linear(input_size, 128)  # Increase neurons in hidden layer
        self.fc2 = nn.Linear(128, 64)  # Additional layer
        self.fc3 = nn.Linear(64, 1)  # Output layer

    def forward(self, x):
        batch_size, seq_len, _ = x.size()
        s = torch.zeros(batch_size, 1)
        v = torch.zeros(batch_size, 1)
        z = torch.zeros(batch_size, 1)  # Initialize adaptation variable `z` to zero
        state = LIFState(v=v, i=s, z=z)  # Add the missing `z` parameter

        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))

        for t in range(seq_len):
            out, state = self.lif1(x[:, t], state)

        out = self.fc3(state.v)
        return out

# Adjust the input shape to match LIFCell
X_train_scaled = X_train_scaled.unsqueeze(1)  # Add a dimension for sequence length
X_test_scaled = X_test_scaled.unsqueeze(1)    # Add a dimension for sequence length

# Training the Enhanced SNN
input_size = X_train_scaled.shape[2]
model = EnhancedSNNRegressor(input_size=input_size)
criterion = nn.MSELoss()
optimizer = Adam(model.parameters(), lr=0.02, weight_decay=1e-5)  # Add weight decay

# Train the model
for epoch in range(100):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_scaled)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch}, Loss {loss.item()}")

# Evaluation
model.eval()
with torch.no_grad():
    predictions = model(X_test_scaled)

    # Convert tensors to NumPy arrays for compatibility with scikit-learn
    y_test_np = y_test.numpy()
    predictions_np = predictions.numpy()

    # Evaluate the model
    mae = mean_absolute_error(y_test_np, predictions_np)
    mse = mean_squared_error(y_test_np, predictions_np)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test_np, predictions_np)

    # Printing Result
    print("MAE:", mae)
    print("Mean Squared Error:", mse)
    print("RMSE:", rmse)
    print("R-squared:", r2)


Epoch 0, Loss 2424271798272.0
Epoch 1, Loss 2424271798272.0
Epoch 2, Loss 2424271536128.0
Epoch 3, Loss 2424271536128.0
Epoch 4, Loss 2424271011840.0
Epoch 5, Loss 2424270749696.0
Epoch 6, Loss 2424270749696.0
Epoch 7, Loss 2424270749696.0
Epoch 8, Loss 2424270487552.0
Epoch 9, Loss 2424270225408.0
Epoch 10, Loss 2424269701120.0
Epoch 11, Loss 2424268914688.0
Epoch 12, Loss 2424268652544.0
Epoch 13, Loss 2424268652544.0
Epoch 14, Loss 2424268914688.0
Epoch 15, Loss 2424268914688.0
Epoch 16, Loss 2424269176832.0
Epoch 17, Loss 2424268914688.0
Epoch 18, Loss 2424268390400.0
Epoch 19, Loss 2424267079680.0
Epoch 20, Loss 2424267079680.0
Epoch 21, Loss 2424266555392.0
Epoch 22, Loss 2424266555392.0
Epoch 23, Loss 2424266817536.0
Epoch 24, Loss 2424266293248.0
Epoch 25, Loss 2424265768960.0
Epoch 26, Loss 2424265244672.0
Epoch 27, Loss 2424265244672.0
Epoch 28, Loss 2424265506816.0
Epoch 29, Loss 2424264982528.0
Epoch 30, Loss 2424264720384.0
Epoch 31, Loss 2424263933952.0
Epoch 32, Loss 242

FF with Backpropagation

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Separate features and target variable
X = df.drop("Total Project Cost (USD)", axis=1)
y = df["Total Project Cost (USD)"]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Reshape X_train to the correct shape (number of samples, number of features)
X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

# Example of further hyperparameter tuning and regularization
model_ffnn = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, input_dim=X_train.shape[1], activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.Dense(1)
])

# Compile with adjusted optimizer and learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model_ffnn.compile(optimizer=optimizer, loss='mean_squared_error')

# Further adjust epochs and batch size
model_ffnn.fit(X_train, y_train, epochs=200, batch_size=64, validation_split=0.1)


# Evaluate the model
loss_ffnn = model_ffnn.evaluate(X_test, y_test)
print(f"FFNN Test Loss: {loss_ffnn}")

# Predicting costs using the FFNN
predictions_ffnn = model_ffnn.predict(X_test)
mae = mean_absolute_error(y_test, predictions_ffnn)
mse = mean_squared_error(y_test, predictions_ffnn)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, predictions_ffnn)

# Printing Result
print("MAE:", mae)
print("Mean Squared Error:", mse)
print("RMSE:", rmse)
print("R-squared:", r2)

Epoch 1/200
96/96 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - loss: 241919754240.0000 - val_loss: 16680259420160.0000
Epoch 2/200
96/96 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 738527477760.0000 - val_loss: 16676299997184.0000
Epoch 3/200
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 447248367616.0000 - val_loss: 16654859763712.0000
Epoch 4/200
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 965605130240.0000 - val_loss: 16602805305344.0000
Epoch 5/200
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 653027704832.0000 - val_loss: 16547384918016.0000
Epoch 6/200
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 211037978624.0000 - val_loss: 16521638182912.0000
Epoch 7/200
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 165765562368.0000 - val_loss: 16515961192448.0000
Epoch 8/200
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 714164469760.0000 - val_loss: 16514816147456.0000
Epoch 9/200
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 1193882550272.0000 - val_loss: 16514447048704.0000
Epoch 10/200
96/9

Elamnn

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import SimpleRNN
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Separate features and target variable
X = df.drop("Total Project Cost (USD)", axis=1)
y = df["Total Project Cost (USD)"]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the Elman-like RNN model
model_enn = tf.keras.models.Sequential([
    SimpleRNN(50, activation='relu', input_shape=(10, 1), return_sequences=False),
    tf.keras.layers.Dense(1)  # Output layer
])

# Compile the model
model_enn.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model_enn.fit(X_train, y_train, epochs=100, batch_size=10, validation_split=0.1)

# Evaluate the model
loss_enn = model_enn.evaluate(X_test, y_test)
print(f"ENN Test Loss: {loss_enn}")

# Predicting costs using the ENN
predictions_enn = model_enn.predict(X_test)
mae = mean_absolute_error(y_test, predictions_enn)
mse = mean_squared_error(y_test, predictions_enn)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, predictions_enn)

# Printing Result
print("MAE:", mae)
print("Mean Squared Error:", mse)
print("RMSE:", rmse)
print("R-squared:", r2)


Epoch 1/100
610/610 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - loss: 728654872576.0000 - val_loss: 16548003577856.0000
Epoch 2/100
610/610 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - loss: 434242191360.0000 - val_loss: 16339376799744.0000
Epoch 3/100
610/610 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 1996036112384.0000 - val_loss: 16366032650240.0000
Epoch 4/100
610/610 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 332363431936.0000 - val_loss: 16382746951680.0000
Epoch 5/100
610/610 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 265485336576.0000 - val_loss: 16475526004736.0000
Epoch 6/100
610/610 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 1650376835072.0000 - val_loss: 16395858345984.0000
Epoch 7/100
610/610 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - loss: 797223485440.0000 - val_loss: 16409353519104.0000
Epoch 8/100
610/610 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 125107937280.0000 - val_loss: 16380623585280.0000
Epoch 9/100
610/610 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 1215825838080.0000 - val_loss: 16372306280448.00

Ensemble Method

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import pandas as pd

# Separate features and target variable
X = df.drop("Total Project Cost (USD)", axis=1)
y = df["Total Project Cost (USD)"]

scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define and train the first model
model1 = MLPRegressor(hidden_layer_sizes=(100, 50), activation='relu', solver='adam', random_state=42)
model1.fit(X_train, y_train)

# Define and train the second model
model2 = MLPRegressor(hidden_layer_sizes=(150, 100, 50), activation='tanh', solver='sgd', random_state=42)
model2.fit(X_train, y_train)

# Define and train the third model
model3 = MLPRegressor(hidden_layer_sizes=(200, 100), activation='relu', solver='adam', alpha=0.01, random_state=42)
model3.fit(X_train, y_train)


MLPRegressor(alpha=0.01, hidden_layer_sizes=(200, 100), random_state=42)

In [ ]:
# Make predictions with each model
y_pred1 = model1.predict(X_test)
y_pred2 = model2.predict(X_test)
y_pred3 = model3.predict(X_test)
# Combine the predictions by averaging
y_pred_ensemble = (y_pred1 + y_pred2 + y_pred3) / 3
# Evaluate the ensemble model
mae_ensemble = mean_absolute_error(y_test, y_pred_ensemble)
mse_ensemble = mean_squared_error(y_test, y_pred_ensemble)
rmse_ensemble = np.sqrt(mse_ensemble)
r2_ensemble = r2_score(y_test, y_pred_ensemble)

# Print the results
print("Ensemble MAE:", mae_ensemble)
print("Ensemble MSE:", mse_ensemble)
print("Ensemble RMSE:", rmse_ensemble)
print("Ensemble R-squared:", r2_ensemble)

Ensemble MAE: 113440.47904572096
Ensemble MSE: 47627894372.30749
Ensemble RMSE: 218238.15975284314
Ensemble R-squared: 0.6720648989465371


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Separate features and target variable
X = df.drop("Total Project Cost (USD)", axis=1)
y = df["Total Project Cost (USD)"]

scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define and train the first model
model1 = MLPRegressor(hidden_layer_sizes=(100, 50), activation='relu', solver='adam', random_state=42)
model1.fit(X_train, y_train)

# Define and train the second model
model2 = MLPRegressor(hidden_layer_sizes=(150, 100, 50), activation='tanh', solver='sgd', random_state=42)
model2.fit(X_train, y_train)

# Define and train the third model
model3 = MLPRegressor(hidden_layer_sizes=(200, 100), activation='relu', solver='adam', alpha=0.01, random_state=42)
model3.fit(X_train, y_train)

# Make predictions with each model
y_pred1 = model1.predict(X_train)
y_pred2 = model2.predict(X_train)
y_pred3 = model3.predict(X_train)

# Stack predictions for the training set of the meta-model
X_meta_train = np.column_stack((y_pred1, y_pred2, y_pred3))

# Meta-model
meta_model = LinearRegression()
meta_model.fit(X_meta_train, y_train)

# Make predictions with each model on the test set
y_pred1_test = model1.predict(X_test)
y_pred2_test = model2.predict(X_test)
y_pred3_test = model3.predict(X_test)

# Stack predictions for the test set of the meta-model
X_meta_test = np.column_stack((y_pred1_test, y_pred2_test, y_pred3_test))

# Meta-model predictions
y_pred_meta = meta_model.predict(X_meta_test)

# Evaluate the stacked model
mae_meta = mean_absolute_error(y_test, y_pred_meta)
mse_meta = mean_squared_error(y_test, y_pred_meta)
rmse_meta = np.sqrt(mse_meta)
r2_meta = r2_score(y_test, y_pred_meta)

# Print the results
print("Stacked Model MAE:", mae_meta)
print("Stacked Model MSE:", mse_meta)
print("Stacked Model RMSE:", rmse_meta)
print("Stacked Model R-squared:", r2_meta)


Stacked Model MAE: 107786.62984078709
Stacked Model MSE: 36897154157.031334
Stacked Model RMSE: 192086.3195467895
Stacked Model R-squared: 0.745949886373592


In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import pandas as pd

# Separate features and target variable
X = df.drop("Total Project Cost (USD)", axis=1)
y = df["Total Project Cost (USD)"]

scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the base models
model1 = MLPRegressor(hidden_layer_sizes=(100, 50), activation='relu', solver='adam', random_state=42)
model2 = MLPRegressor(hidden_layer_sizes=(150, 100, 50), activation='tanh', solver='sgd', random_state=42)
model3 = MLPRegressor(hidden_layer_sizes=(200, 100), activation='relu', solver='adam', alpha=0.01, random_state=42)

# Initialize the meta-features array
meta_features = np.zeros((X_train.shape[0], 3))  # Adjust number of columns to match the number of base models
meta_targets = y_train

# Perform cross-validation to create meta-features
kf = KFold(n_splits=5, shuffle=True, random_state=42)
for train_idx, val_idx in kf.split(X_train):
    X_train_fold, X_val_fold = X_train[train_idx], X_train[val_idx]
    # Use .iloc to slice y_train based on integer position
    y_train_fold, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]

    # Train each model on the training fold
    model1.fit(X_train_fold, y_train_fold)
    model2.fit(X_train_fold, y_train_fold)
    model3.fit(X_train_fold, y_train_fold)

    # Generate meta-features for the validation fold
    meta_features[val_idx, 0] = model1.predict(X_val_fold)
    meta_features[val_idx, 1] = model2.predict(X_val_fold)
    meta_features[val_idx, 2] = model3.predict(X_val_fold)

# Train the meta-model on the meta-features
meta_model = LinearRegression()
meta_model.fit(meta_features, meta_targets)

# Generate meta-features for the test set
X_meta_test = np.column_stack((model1.predict(X_test), model2.predict(X_test), model3.predict(X_test)))

# Meta-model predictions
y_pred_meta = meta_model.predict(X_meta_test)

# Evaluate the stacked model
mae_meta = mean_absolute_error(y_test, y_pred_meta)
mse_meta = mean_squared_error(y_test, y_pred_meta)
rmse_meta = np.sqrt(mse_meta)
r2_meta = r2_score(y_test, y_pred_meta)

# Print the results
print("Stacked Model MAE:", mae_meta)
print("Stacked Model MSE:", mse_meta)
print("Stacked Model RMSE:", rmse_meta)
print("Stacked Model R-squared:", r2_meta)

Stacked Model MAE: 66475.08064520282
Stacked Model MSE: 24247347105.265633
Stacked Model RMSE: 155715.59685935648
Stacked Model R-squared: 0.8330483358956345


Augmentation

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Separate features and target variable
X = df.drop("Total Project Cost (USD)", axis=1)
y = df["Total Project Cost (USD)"]

scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def add_gaussian_noise(X, y, noise_level=0.01):
    noise = np.random.normal(0, noise_level, X.shape)
    X_noisy = X + noise
    return X_noisy, y

def transform_data(X, y):
    # Handle potential negative or zero values before log transformation
    X_non_negative = np.where(X <= 0, 1e-10, X)  # Replace non-positive values with a small positive value
    X_transformed = np.log1p(X_non_negative)  # Apply log transformation
    return X_transformed, y

def data_augmentation_pipeline(X, y):
    # Step 1: Add Gaussian noise
    X_noisy, y_noisy = add_gaussian_noise(X, y)

    # Step 2: Apply log transformation
    X_transformed, y_transformed = transform_data(X, y)

    # Combine original and augmented data
    X_augmented = np.vstack((X, X_noisy, X_transformed))
    y_augmented = np.hstack((y, y_noisy, y_transformed))

    return X_augmented, y_augmented

# Apply the data augmentation pipeline
X_train_augmented, y_train_augmented = data_augmentation_pipeline(X_train, y_train)

# Define and train the MLPNN model with augmented data
mlp = MLPRegressor(hidden_layer_sizes=(100, 50), activation='relu', solver='adam', random_state=42)
mlp.fit(X_train_augmented, y_train_augmented)

# Make predictions
y_pred_augmented = mlp.predict(X_test)

# Evaluate the model
mae_augmented = mean_absolute_error(y_test, y_pred_augmented)
mse_augmented = mean_squared_error(y_test, y_pred_augmented)
rmse_augmented = np.sqrt(mse_augmented)
r2_augmented = r2_score(y_test, y_pred_augmented)

# Printing Result
print("MAE with Augmented Data:", mae_augmented)
print("Mean Squared Error with Augmented Data:", mse_augmented)
print("RMSE with Augmented Data:", rmse_augmented)
print("R-squared with Augmented Data:", r2_augmented)

MAE with Augmented Data: 102948.81284347716
Mean Squared Error with Augmented Data: 35122481867.61503
RMSE with Augmented Data: 187409.93001336677
R-squared with Augmented Data: 0.7581691403262694
